In [17]:
import snap
from os import path
import pickle

In [18]:
BASE_PATH = "../data/academia.stackexchange.com"
FOLDED_NGRAM_GRAPH_PATH = path.join(BASE_PATH, "Userid_Ngram_Folded_Graph_Sept2017.graph")
FOLDED_POSTID_GRAPH_PATH = path.join(BASE_PATH, "Postid_Folded_Graph_Sept2017.graph")

NGRAM_DICT_PICKLE = path.join(BASE_PATH, "PostSept2017/Ngramid_Dict_PostSept17.pickle")

In [3]:
# f_in = snap.TFIn(FOLDED_POSTID_GRAPH_PATH)
# post_graph = snap.TUNGraph.Load(f_in)
# print "nodes", post_graph.GetNodes()
# print "edges", post_graph.GetEdges()

# labels = snap.TIntStrH()
# for NI in post_graph.Nodes():
#     labels[NI.GetId()] = str(NI.GetId())
# snap.DrawGViz(post_graph, snap.gvlDot, "post-output.png", "Folded Post Graph", labels)

nodes 15624
edges 1970851


In [19]:
f_in = snap.TFIn(FOLDED_NGRAM_GRAPH_PATH)
ngram_graph = snap.TUNGraph.Load(f_in)
print "nodes", ngram_graph.GetNodes()
print "edges", ngram_graph.GetEdges()

# labels = snap.TIntStrH()
# for NI in user_graph.Nodes():
#     labels[NI.GetId()] = str(NI.GetId())
# snap.DrawGViz(user_graph, snap.gvlDot, "ngram-output.png", "Folded Ngram Graph", labels)

nodes 17907
edges 5469050


In [20]:
COMMUNITIES_PATH = path.join(BASE_PATH, 'ngram-communities.txt')
COMMUNITIES_VEC_PATH = path.join(BASE_PATH, 'ngram-communities.vector')

# CNM
assert snap.CntSelfEdges(ngram_graph) == 0

comm_vec = snap.TCnComV()
modularity = snap.CommunityCNM(ngram_graph, comm_vec)

f_out = snap.TFOut(COMMUNITIES_VEC_PATH)
comm_vec.Save(f_out)
f_out.Flush()

In [24]:
f_in = snap.TFIn(COMMUNITIES_VEC_PATH)
comm_vec = snap.TCnComV()
comm_vec.Load(f_in)

print "communities", len(comm_vec)

pickle_file = open(NGRAM_DICT_PICKLE, 'rb')
ngram_dict = pickle.load(pickle_file)
inverted_dict = dict([[v,k] for k,v in ngram_dict.items()])

with open(COMMUNITIES_PATH, 'w') as f:
    for i, comm in enumerate(comm_vec):
        f.write("#####Community {}#####\n".format(i))
        for node in comm:
            f.write(inverted_dict[node] + '\n')
    f.write("The modularity of the network is {}".format(modularity))

communities 324


In [25]:
# modularity of communities
f_in = snap.TFIn(COMMUNITIES_VEC_PATH)
comm_vec = snap.TCnComV()
comm_vec.Load(f_in)

for i, comm in enumerate(comm_vec):
    community = snap.TIntV()
    for node in comm:
        community.Add(node)
    print 'Community {} modularity: {}'.format(i, snap.GetModularity(ngram_graph, community, ngram_graph.GetEdges()))

Community 0 modularity: 0.0794793553649
Community 1 modularity: 0.076593659917
Community 2 modularity: 0.042926670411
Community 3 modularity: 5.48539197445e-07
Community 4 modularity: 0.000753770334576
Community 5 modularity: 0.00100481321339
Community 6 modularity: 0.0122501731429
Community 7 modularity: 1.09708053461e-06
Community 8 modularity: 5.48540133571e-07
Community 9 modularity: 1.09708053461e-06
Community 10 modularity: 5.04393206868e-05
Community 11 modularity: 4.0224005688e-06
Community 12 modularity: 5.48539197445e-07
Community 13 modularity: 0.000193449093429
Community 14 modularity: 5.48540133571e-07
Community 15 modularity: 5.48540133571e-07
Community 16 modularity: 5.48540133571e-07
Community 17 modularity: 8.22779237993e-06
Community 18 modularity: 1.09707184201e-06
Community 19 modularity: 1.09707933102e-06
Community 20 modularity: 2.19406785782e-06
Community 21 modularity: 2.90632995211e-05
Community 22 modularity: 8.42283403532e-05
Community 23 modularity: 1.097080